In [ ]:
from valurap import gcode, asg, path_planning
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import IPython
import time
from IPython.display import display
import IPython
import os
import pickle

import imp
imp.reload(path_planning)
imp.reload(gcode)
imp.reload(asg)


max_ae = 1000

acc_step = 10000
v_step = 50000000
v_mult = v_step / acc_step
spm = 80
spme = 837
apg_z = path_planning.FakeApg("Z")

def ext_to_code(e, f):
    segs = []
    abs_e = abs(e)
    acc_dt = f / max_ae
    full_de = max_ae * acc_dt * acc_dt # / 2 * 2 
    if abs_e > full_de:
        plato_de = abs_e - full_de
        plato_dt = plato_de / f
        plato_v = f
    else:
        acc_dt = mth.sqrt(abs_e / max_ae)
        plato_v = max_ae * acc_dt
        plato_dt = 0
    
    int_acc_dt = int(acc_dt * acc_step)
    int_plato_dt = int(plato_dt * acc_step)
    int_ve = int(plato_v * 2 ** 32 * spme / v_step)
    int_ae = int(int_ve / int_acc_dt) * 65536
    
    if e < 0:
        int_ve = -int_ve
        int_ae = -int_ae

    segs.append([int_acc_dt, [asg.ProfileSegment(apg = apg_z, v=0, a = int_ae)]])
    if int_plato_dt > 0:
        segs.append([int_plato_dt, [asg.ProfileSegment(apg = apg_z, v=int_ve, a=0)]])
    segs.append([int_acc_dt, [asg.ProfileSegment(apg = apg_z, v=int_ve, a = -int_ae)]])
    segs.append([1, [asg.ProfileSegment(apg = apg_z, v=0, a=0)]])
    return segs        

def segment_to_code(seg):
    seg = np.array(seg)
    t0 = time.time()
    path, slowdowns = path_planning.make_path(seg)
    corner_errors_slowdowns, cc = path_planning.process_corner_errors(path, slowdowns)
    corner_space_slowdowns, cc = path_planning.process_corner_space(path, corner_errors_slowdowns)
    plato_slowdowns = corner_space_slowdowns
    for i in range(20):
        new_plato_slowdowns, stage_ok, cc, nc, sc = path_planning.process_plato(path, plato_slowdowns)
        plato_slowdowns = new_plato_slowdowns
        print("Result", i, len(sc[sc["slowdown"] < 0.999]))

        if stage_ok:
            break

    final_slowdowns, stage_ok, cc1, nc1, sc1 = path_planning.process_plato(path, plato_slowdowns)
    ce_slowdowns, ce1 = path_planning.process_corner_errors(path, plato_slowdowns)
    cs_slowdowns, cs1 = path_planning.process_corner_space(path, plato_slowdowns)
    speeds = path_planning.gen_speeds(path, plato_slowdowns)

    if not stage_ok:
        print("WARNING: Plato stage is not finished!!!")
        
    t1 = time.time()
    print("Planning time:", t1 - t0)
    
    all_segments = path_planning.build_segments(path, plato_slowdowns)
    return path_planning.format_segments(all_segments, acc_step=acc_step)


logging.basicConfig(level=logging.DEBUG)
logging.getLogger('valurp.path_planning').setLevel(logging.DEBUG)

fn = "test4"
lines = gcode.reader(fn + ".gcode")
pg = gcode.path_gen(lines)
sg = gcode.gen_segments(pg)
todo = []
layer_num = 0
layer_data = []
current_segment = []
fn_tpl = fn + "_{:05d}.layer"
for i, s in enumerate(sg):
    #print(str(s)[:100])
    if isinstance(s, gcode.do_move):
        print(i, s)
        if "Z" in s.deltas:
            tupled_segment = []
            for dt, segs in current_segment:
                tupled_segment.append((dt, tuple([s.to_tuple() for s in segs])))
                
            layer_data.append(("segment", tupled_segment))
            with open(fn_tpl.format(layer_num) + ".tmp", "wb") as f:
                pickle.dump(layer_data, f)
            os.rename(fn_tpl.format(layer_num) + ".tmp", fn_tpl.format(layer_num))
            layer_num += 1
            layer_data = [("start", s.target, s.deltas)]
            current_segment = []
        else:
            assert(False)
    elif isinstance(s, gcode.do_ext):
        print(i, s)
        current_segment.extend(ext_to_code(s.deltas["E"], s.deltas["F"]/60.0))
    elif isinstance(s, gcode.do_home):
        print(i, s)
        if current_segment:
            tupled_segment = []
            for dt, segs in current_segment:
                tupled_segment.append((dt, tuple([s.to_tuple() for s in segs])))
                
            layer_data.append(("segment", tupled_segment))
            current_segment = []
        layer_data.append(("do_home",))
    elif isinstance(s, gcode.do_segment):
        current_segment.extend(segment_to_code(s.path))

        print("segment", i, len(s.path))
        #if i > 135:
        #    break
        #elif i >= 10 and i <= 225:
        #    todo.append(s)
    else:
        assert(False)

if current_segment:
    tupled_segment = []
    for dt, segs in current_segment:
        tupled_segment.append((dt, tuple([s.to_tuple() for s in segs])))
    layer_data.append(("segment", tupled_segment))
    current_segment = []

if layer_data:
    with open(fn_tpl.format(layer_num) + ".tmp", "wb") as f:
        pickle.dump(layer_data, f)
    os.rename(fn_tpl.format(layer_num) + ".tmp", fn_tpl.format(layer_num))
    
#seg = np.array(todo[0].path)

In [ ]:
from valurap import path_planning, emulate
import imp
imp.reload(emulate)
imp.reload(path_planning)

path_planning.gen_layers("test4")

In [ ]:
seg = np.array(s.path)

In [ ]:
a = 2
b = 5
c = 100
seg = [
    [0, 0, 0],
    [b, 0, 100],
    [b, 0.01, 300],
    [b, 0.02, 300],
    [b + 1, a, 300],
    [b + c, a, 200],
    [b + c, a, 0]
]

seg = np.array(seg)

In [ ]:
import numpy as np
import pandas as pd
import IPython

path, slowdowns = path_planning.make_path(seg)
IPython.display.display(path)
IPython.display.display(slowdowns)


In [ ]:
speeds = path_planning.gen_speeds(path, slowdowns)
speeds[0:1]

In [ ]:
# corner calculations

corner_errors_slowdowns, cc = path_planning.process_corner_errors(path, slowdowns)
cc
corner_errors_slowdowns[73:76]

In [ ]:
import IPython
new_slowdowns, cc = path_planning.process_corner_errors(path, corner_errors_slowdowns)
IPython.display.display(cc)
IPython.display.display(new_slowdowns)

In [ ]:
corner_space_slowdowns, cc = path_planning.process_corner_space(path, corner_errors_slowdowns)
speeds = path_planning.gen_speeds(path, corner_space_slowdowns)

IPython.display.display(cc)
IPython.display.display(corner_space_slowdowns)
IPython.display.display(speeds)

In [ ]:
plato_slowdowns = corner_space_slowdowns
for i in range(20):
    new_plato_slowdowns, stage_ok, cc, ac, dc, nc, sc = path_planning.process_plato(path, plato_slowdowns)
    plato_slowdowns = new_plato_slowdowns
    print("Result", i, len(sc[sc["slowdown"] < 0.999]))

    if stage_ok:
        break            

speeds = path_planning.gen_speeds(path, plato_slowdowns)

IPython.display.display(cc)
IPython.display.display(ac)
IPython.display.display(dc)
IPython.display.display(sc)
IPython.display.display(nc)
IPython.display.display(plato_slowdowns)
IPython.display.display(speeds)

plato2_slowdowns, stage_ok, cc1, ac1, dc1, nc1, sc1 = path_planning.process_plato(path, plato_slowdowns)
IPython.display.display(sc1)
IPython.display.display(nc1)
IPython.display.display(plato2_slowdowns)


In [ ]:
######## All-in-One
imp.reload(path_planning)
t0 = time.time()
path, slowdowns = path_planning.make_path(seg)
corner_errors_slowdowns, cc = path_planning.process_corner_errors(path, slowdowns)
corner_space_slowdowns, cc = path_planning.process_corner_space(path, corner_errors_slowdowns)
plato_slowdowns = corner_space_slowdowns
for i in range(20):
    new_plato_slowdowns, stage_ok, cc, nc, sc = path_planning.process_plato(path, plato_slowdowns)
    plato_slowdowns = new_plato_slowdowns
    print("Result", i, len(sc[sc["slowdown"] < 0.995]))
    #display(sc[sc["slowdown"] < 0.995])

    if stage_ok:
        break

final_slowdowns, stage_ok, cc1, nc1, sc1 = path_planning.process_plato(path, plato_slowdowns)
ce_slowdowns, ce1 = path_planning.process_corner_errors(path, plato_slowdowns)
cs_slowdowns, cs1 = path_planning.process_corner_space(path, plato_slowdowns)
speeds = path_planning.gen_speeds(path, plato_slowdowns)

assert stage_ok
t1 = time.time()
print("Planning time:", t1 - t0)


In [ ]:
print("---- path ----")
display(path.iloc[0])
print("---- ce1 ----")
display(ce1.iloc[0])
print("---- cs1 ----")
display(cs1.iloc[0])
print("---- nc1 ----")
display(nc1.iloc[0])
print("---- speeds ----")
display(speeds.iloc[0])
l = len(cs1)
print("---- cs1 ----")
display(cs1.iloc[l-10:])
print("---- path ----")
display(path.iloc[l-10:])

In [ ]:
%matplotlib notebook

from matplotlib.collections import LineCollection
import warnings

imp.reload(path_planning)

warnings.simplefilter("error")

all_segments = path_planning.build_segments(path, plato_slowdowns)

lines = []
colors = []

i = 0
for index, row in all_segments.iterrows():
    i += 1
    lines.append(((row['x0'], row['y0']), (row['x1'], row['y1'])))
    st = row["seg_type"]
    if st == "start":
        colors.append((1,0,0,1))
    elif st == "end":
        colors.append((0,1,0,1))
    elif st == "short_plato":
        colors.append((0,0.3,1,1))
    elif st == "middle_plato":
        colors.append((0.8,0,0.8,1))
    elif st == "long_plato_main":
        colors.append((0.2,0.2,0.2,1))
    elif st == "long_plato_in":
        colors.append((1,0,0.8,1))
    elif st == "long_plato_out":
        colors.append((0,1,0.8,1))
    else:
        colors.append((1,1,0,1))
    
lc = LineCollection(lines, colors=colors, linewidths=1)
fig, ax = plt.subplots()

ax.add_collection(lc)

#path.iloc[limit:].plot(x="px", y="py", linewidth=3, color=(0.5, 0.5, 0.5, 0.2), ax=ax)
full_path = pd.DataFrame()
full_path["x"] = path["px"]
full_path["y"] = path["py"]
last_x = path["x"].iloc[-1]
last_y = path["y"].iloc[-1]
full_path = full_path.append({"x": last_x, "y": last_y}, ignore_index=True)
full_path.plot(x="x", y="y", linewidth=5, color=(0.5, 0.5, 0.5, 0.2), ax=ax)
    
ax.autoscale()
ax.margins(0.1)
display(full_path.iloc[:10])
display(all_segments.iloc[:10])

display(full_path.iloc[-10:])
display(all_segments.iloc[-10:])


In [ ]:
all_segments["test_vx1"] = all_segments["vx0"] + all_segments["dt"] * all_segments["ax0"]
all_segments["test_vy1"] = all_segments["vy0"] + all_segments["dt"] * all_segments["ay0"]
all_segments["test_x1"] = all_segments["x0"] + all_segments["dt"] * all_segments["vx0"] + all_segments["dt"] * all_segments["dt"] * all_segments["ax0"]/2
all_segments["test_y1"] = all_segments["y0"] + all_segments["dt"] * all_segments["vy0"] + all_segments["dt"] * all_segments["dt"] * all_segments["ay0"]/2
print("bad vx1")
display(all_segments[abs(all_segments["test_vx1"] - all_segments["vx1"]) > 0.01])
print("bad vy1")
display(all_segments[abs(all_segments["test_vy1"] - all_segments["vy1"]) > 0.01])
print("bad x1")
display(all_segments[abs(all_segments["test_x1"] - all_segments["x1"]) > 0.0001])
print("bad y1")
display(all_segments[abs(all_segments["test_y1"] - all_segments["y1"]) > 0.0001])
print("ax non continuous")
display(all_segments[abs(all_segments["ax1"] - all_segments["ax0"]) > 0.001])
print("ay non continuous")
display(all_segments[abs(all_segments["ay1"] - all_segments["ay0"]) > 0.001])
print("vx non continuous")
display(all_segments[abs(all_segments["vx1"] - all_segments["vx0"].shift(-1)) > 0.001])
print("vy non continuous")
display(all_segments[abs(all_segments["vy1"] - all_segments["vy0"].shift(-1)) > 0.001])
print("x non continuous")
display(all_segments[abs(all_segments["x1"] - all_segments["x0"].shift(-1)) > 0.001])
print("y non continuous")
display(all_segments[abs(all_segments["y1"] - all_segments["y0"].shift(-1)) > 0.001])
print("src_part groups")
display(np.bincount(all_segments["src_idx"], weights=all_segments["src_part"]))
display(all_segments[np.isnan(all_segments["dt"])][["seg_type", "vx0", "vx1", "vy0", "vy1"]])


In [ ]:
sub_path = pd.DataFrame()
sub_path["src_idx"] = path["src_idx"]
sub_path["src_de"] = path["de"]
all_segments_extras = all_segments.merge(sub_path, on='src_idx')
all_segments_extras["de"] = all_segments_extras["src_de"] * all_segments_extras["src_part"]
all_segments_extras["de_v"] = all_segments_extras["de"] / all_segments_extras["dt"]
all_segments_extras.loc[abs(all_segments_extras["de"]) < 0.001, 'de_v' ] = 0
all_segments_extras["v0"] = np.linalg.norm(all_segments_extras[['vx0', 'vy0']].values, axis=1)
all_segments_extras["v1"] = np.linalg.norm(all_segments_extras[['vx1', 'vy1']].values, axis=1)
all_segments_extras["avg_v"] = (all_segments_extras["v0"] + all_segments_extras["v1"])/2
all_segments_extras["de_v0"] = all_segments_extras["de_v"] / all_segments_extras["avg_v"] * all_segments_extras["v0"]
all_segments_extras["de_v1"] = all_segments_extras["de_v"] / all_segments_extras["avg_v"] * all_segments_extras["v1"]
all_segments_extras["de_a"] = (all_segments_extras["de_v1"] - all_segments_extras["de_v0"])/ all_segments_extras["dt"]
all_segments_extras.loc[abs(all_segments_extras["de"]) < 0.001, "de_v0"] = 0
all_segments_extras.loc[abs(all_segments_extras["de"]) < 0.001, "de_v1"] = 0
all_segments_extras.loc[abs(all_segments_extras["de"]) < 0.001, "de_a"] = 0

acc_step = 10000
v_step = 50000000
v_mult = v_step / acc_step
spm = 80

delta_e = all_segments_extras["de_v1"] - all_segments_extras["de_v0"].shift(-1)
all_segments_extras["delta_ve"] = delta_e
all_segments_extras["int_dt"] = (all_segments_extras["dt"] * acc_step).fillna(0).astype(int)
all_segments_extras["int_vx0"] = (all_segments_extras["vx0"] * 2 ** 32 * spm / v_step).fillna(0).astype(int)
all_segments_extras["int_vy0"] = (all_segments_extras["vy0"] * 2 ** 32 * spm / v_step).fillna(0).astype(int)
all_segments_extras["int_vx1"] = (all_segments_extras["vx1"] * 2 ** 32 * spm / v_step).fillna(0).astype(int)
all_segments_extras["int_vy1"] = (all_segments_extras["vy1"] * 2 ** 32 * spm / v_step).fillna(0).astype(int)
all_segments_extras["int_ax"] = (np.around((all_segments_extras["int_vx1"] - all_segments_extras["int_vx0"]) / all_segments_extras["int_dt"])).replace([np.inf, -np.inf], np.nan).fillna(0).astype(int) 
all_segments_extras["int_ay"] = (np.around((all_segments_extras["int_vy1"] - all_segments_extras["int_vy0"]) / all_segments_extras["int_dt"])).replace([np.inf, -np.inf], np.nan).fillna(0).astype(int) 
#all_segments_extras["int_vx1_real1"] = (all_segments_extras["int_vx0"] + (all_segments_extras["int_ax"]-1) * all_segments_extras["int_dt"]).fillna(0).astype(int)
all_segments_extras["int_vx1_real"] = (
    all_segments_extras["int_vx0"] + all_segments_extras["int_ax"] * all_segments_extras["int_dt"]
).fillna(0).astype(int)
#all_segments_extras["int_vx1_real3"] = (all_segments_extras["int_vx0"] + (all_segments_extras["int_ax"]+1) * all_segments_extras["int_dt"]).fillna(0).astype(int)
#all_segments_extras["int_vy1_real1"] = (all_segments_extras["int_vy0"] + (all_segments_extras["int_ay"]-1) * all_segments_extras["int_dt"]).fillna(0).astype(int)
all_segments_extras["int_vy1_real"] = (
    all_segments_extras["int_vy0"] + all_segments_extras["int_ay"] * all_segments_extras["int_dt"]
).fillna(0).astype(int)
#all_segments_extras["int_vy1_real3"] = (all_segments_extras["int_vy0"] + (all_segments_extras["int_ay"]+1) * all_segments_extras["int_dt"]).fillna(0).astype(int)
all_segments_extras["int_dx"] = ((all_segments_extras["int_vx0"] + all_segments_extras["int_vx1_real"]) / 2 * all_segments_extras["int_dt"] * v_mult).fillna(0).astype(int)
all_segments_extras["int_dy"] = ((all_segments_extras["int_vy0"] + all_segments_extras["int_vy1_real"]) / 2 * all_segments_extras["int_dt"] * v_mult).fillna(0).astype(int)
all_segments_extras["real_dx"] = all_segments_extras["int_dx"] / spm / 2.0 ** 32
all_segments_extras["exp_dx"] = all_segments_extras["x1"] - all_segments_extras["x0"]
all_segments_extras["real_dy"] = all_segments_extras["int_dy"] / spm / 2.0 ** 32
all_segments_extras["exp_dy"] = all_segments_extras["y1"] - all_segments_extras["y0"]
all_segments_extras["error"] = np.linalg.norm([
    all_segments_extras["real_dx"] - all_segments_extras["exp_dx"],
    all_segments_extras["real_dy"] - all_segments_extras["exp_dy"],
], axis = 0)

#display(all_segments_extras[all_segments_extras["error"]>0.02])
display(all_segments_extras[np.logical_and(all_segments_extras["dt"] > -1e-27,all_segments_extras["dt"] < 1e-4)][["seg_type","dt", "vx0", "vx1", "vy0", "vy1", "exp_dx", "exp_dy"]])


In [ ]:
from valurap.asg import ProfileSegment
from numpy.linalg import norm
imp.reload(path_planning)

acc_step = 10000
pr_opt = path_planning.format_segments(all_segments, acc_step=acc_step)
display(pr_opt)
res = path_planning.emulate(pr_opt, accel_step=50000000/acc_step, verbose=0)



In [ ]:
import pickle
with open("plan.pkl", "wb") as f:
    pickle.dump(pr_opt, f)

In [ ]:
spm = 80
spme = 847
res["x"] = res["X_x"] / spm
res["y"] = res["Y_x"] / spm
res["xv"] = res["X_v"] * 65536 / 2**32 / spm * 50000000
res["yv"] = res["Y_v"] * 65536 / 2**32 / spm * 50000000
res["ev"] = res["Z_v"] * 65536 / 2**32 / spme * 50000000
res[["x", "y", "xv", "yv", "ts"]]


lines = []
colors = []

i = 0
for index, row in all_segments.iterrows():
    i += 1
    lines.append(((row['x0'], row['y0']), (row['x1'], row['y1'])))
    st = row["seg_type"]
    if st == "start":
        colors.append((1,0,0,1))
    elif st == "end":
        colors.append((0,1,0,1))
    elif st == "short_plato":
        colors.append((0,0.3,1,1))
    elif st == "middle_plato":
        colors.append((0.8,0,0.8,1))
    elif st == "long_plato_main":
        colors.append((0.2,0.2,0.2,1))
    elif st == "long_plato_in":
        colors.append((1,0,0.8,1))
    elif st == "long_plato_out":
        colors.append((0,1,0.8,1))
    else:
        colors.append((1,1,0,1))
    
lc = LineCollection(lines, colors=colors, linewidths=1)
fig, ax = plt.subplots()

ax.add_collection(lc)

#path.iloc[limit:].plot(x="px", y="py", linewidth=3, color=(0.5, 0.5, 0.5, 0.2), ax=ax)
full_path = pd.DataFrame()
full_path["x"] = path["px"]
full_path["y"] = path["py"]
last_x = path["x"].iloc[-1]
last_y = path["y"].iloc[-1]
full_path = full_path.append({"x": last_x, "y": last_y}, ignore_index=True)
full_path.plot(x="x", y="y", linewidth=5, color=(0.5, 0.5, 0.5, 0.2), ax=ax)
res.plot(x="x", y="y", style="-", ax=ax)
    
ax.autoscale()
ax.margins(0.1)

In [ ]:
res.plot(x="t", y=["xv", "yv", "ev"], secondary_y = ["ev"])



In [ ]:
int_dt = 80.0
int_dx = -91527257967.38159
int_dy = -25116203791.99707

print("expected dx:", int_dx / 2**32 / spm)

int_vx0 = -115446.0 
int_vx1 = -170602.0

int_vy0 = -30079.0
int_vy1 = 0.0

int_ax = -45183795.0
int_ay = 24640717.0

v_step = 50000000
v_mult = v_step / acc_step
spm = 80
    
real_dx = path_planning.int_x(int_dt, int_vx0 * 65536, int_ax, 0, 0) / 65536 * v_mult
print("real_dx:", real_dx, (real_dx - int_dx) / 2**32/spm)
x_error = int_dx - real_dx

apg = path_planning.FakeApg("X")

old_segs = [
    ProfileSegment(
        apg=apg, x=0, v=int(int_vx0), a=int(int_ax), j=0
    ),
]

old_profile = [[int(int_dt), old_segs]]

if 1:
    int_jx = np.around(-12 * x_error / int_dt / int_dt / int_dt * 65536 / v_mult)
    int_ax_new = np.around(int_ax - int_jx * int_dt / 2)
    
print("int_jx:", int_jx, "int_ax:", int_ax, "int_ax_new:", int_ax_new)



new_segs = [
    ProfileSegment(
        apg=apg, x=0, v=int(int_vx0), a=int(int_ax_new), j=int(int_jx)
    ),
]

new_profile = [[int(int_dt), new_segs]]

old_res = path_planning.emulate(old_profile, accel_step=50000000 / acc_step)
new_res = path_planning.emulate(new_profile, accel_step=50000000 / acc_step)


real_dx_new = path_planning.int_x(int_dt, int_vx0 * 65536, int_ax_new, int_jx, 0) / 65536 * v_mult
print("real dx new:", real_dx_new, (real_dx_new - int_dx) / 2**32/spm)

display(old_res)
display(new_res)

In [ ]:
import pickle
import imp
from valurap import asg, path_planning, emulate
imp.reload(asg)
imp.reload(path_planning)

with open("test4_00001.layer", "rb") as f:
    p = pickle.load(f)
assert len(p) == 2
assert p[0][0] == "start"
assert p[1][0] == "segment"

spm = 80
spme = 847
acc_step = 10000

apgs = {
    "X": path_planning.FakeApg("X"),
    "Y": path_planning.FakeApg("Y"),
    "Z": path_planning.FakeApg("Z"),
}

pr_opt = [
    [1, [
        asg.ProfileSegment(apgs["X"], x=p[0][1]["X"] * spm),
        asg.ProfileSegment(apgs["Y"], x=p[0][1]["Y"] * spm),
        asg.ProfileSegment(apgs["Z"], x=0),
    ]]
]
for dt, segs in p[1][2]:
    pr_opt.append([
        dt, [asg.ProfileSegment.from_tuple(s, apgs) for s in segs]
    ])

display(pr_opt)
res = emulate.emulate(pr_opt, accel_step=50000000/acc_step, verbose=0)
res["x"] = res["X_x"] / spm
res["y"] = res["Y_x"] / spm
res["e"] = res["Z_x"] / spme
res["xv"] = res["X_v"] * 65536 / 2**32 / spm * 50000000
res["yv"] = res["Y_v"] * 65536 / 2**32 / spm * 50000000
res["ev"] = res["Z_v"] * 65536 / 2**32 / spme * 50000000
display(res)



In [ ]:
res.plot(x="t", y=["xv", "yv", "ev"], secondary_y = ["ev"])


In [ ]:
%matplotlib notebook
res.plot(x="x", y="y")